In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from feed_forward import canteen_model, plot_training_set, plot_history, preprocess, normalize_dataset, calculate_optimized_parameters, predict_canteen_values

In [ ]:
train_dataset, test_dataset, train_labels, test_labels = preprocess(pd.read_csv("../../data/ml_df.csv", index_col="date"))
normed_train_data, normed_test_data = normalize_dataset(train_dataset, test_dataset)

In [ ]:
plot_training_set(pd.merge(train_labels,train_dataset, left_index=True, right_index=True))

In [ ]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

In [ ]:
history, model = canteen_model(normed_train_data, train_labels)

In [ ]:
model.summary()

In [ ]:
# Testing that model is working
example_batch = train_dataset[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
plot_history(history)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
print("Testing set Mean Abs Error: {:5.2f} canteen visitors".format(mae))

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(list(test_labels["Canteen"]), test_predictions, c='b', label="prediction")
plt.xlabel('True Values [Canteen]')
plt.ylabel('Predictions [Canteen]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
plt.plot([-10000, 10000], [-10000, 10000], c='r', label="actual")
plt.legend(loc="best")
plt.show()

In [ ]:
error = test_predictions - list(test_labels["Canteen"])
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [Canteen]")
plt.ylabel("Count")
plt.show()

In [ ]:
# Creates csv file with results from trying different parameters
# calculate_optimized_parameters(train_dataset, train_labels)

In [ ]:
test_prediction = pd.read_csv("../../data/ml_df.csv", index_col="date")
res, canteen_dates = predict_canteen_values(test_prediction)

In [ ]:

canteen_dates['feed_forward_pred'] = res
canteen_dates = canteen_dates.drop(["Canteen"], axis=1)
canteen_dates